# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [25]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [26]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lavras da mangabeira,-6.7533,-38.9644,308.39,23,54,3.38,BR,1702323693
1,1,adamstown,-25.0660,-130.1015,294.70,61,1,2.48,PN,1702323693
2,2,waitangi,-43.9535,-176.5597,287.49,88,100,3.13,NZ,1702323693
3,3,papatowai,-46.5619,169.4708,281.92,70,54,8.90,NZ,1702323693
4,4,stanley,54.8680,-1.6985,275.24,100,0,1.54,GB,1702323409


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [27]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.scatter(
    x='Lng',
    y='Lat',
    c='Humidity',
    cmap='viridis',  # You can change the colormap as needed
    title='City Map with Humidity',
    colorbar=True,
    size='Humidity',
    frame_width=700,
    frame_height=500)

# Display the map
map_plot

:Scatter   [Lng]   (Lat,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [28]:
# Narrow down cities that fit criteria and drop any results with null values
perfect_conditions = (
    (city_data_df['Max Temp'] < 250) & 
    (city_data_df['Max Temp'] > 240))

perfect_cities_df = city_data_df[perfect_conditions]

# Drop any rows with null values
perfect_cities_df = perfect_cities_df.dropna()

# Display sample data
perfect_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
15,15,tazovsky,67.4667,78.7000,248.11,95,100,4.22,RU,1702323694
24,24,udachny,66.4167,112.4000,247.84,97,100,4.50,RU,1702323695
26,26,jalai nur,49.4500,117.7000,248.05,98,100,1.00,CN,1702323695
42,42,erdenet,49.0333,104.0833,246.27,100,65,0.43,MN,1702323697
56,56,dudinka,69.4058,86.1778,248.53,98,100,6.26,RU,1702323698


### Step 3: Create a new DataFrame called `hotel_df`.

In [31]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = perfect_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
15,tazovsky,RU,67.4667,78.7000,95,
24,udachny,RU,66.4167,112.4000,97,
26,jalai nur,CN,49.4500,117.7000,98,
42,erdenet,MN,49.0333,104.0833,100,
56,dudinka,RU,69.4058,86.1778,98,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [36]:
# Set parameters to search for a hotel
radius = 10000  
params = {
    "radius": radius,
    "format": "json",
    "apiKey": "8b7e9c177f284f429b8ee7535216643f"}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    Lat = row["Lat"]
    Lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"lat:{Lat} lng:{Lng}"
    params["bias"] = f"lat:{Lat} lng:{Lng}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()


Starting hotel search
tazovsky - nearest hotel: No hotel found
udachny - nearest hotel: No hotel found
jalai nur - nearest hotel: No hotel found
erdenet - nearest hotel: No hotel found
dudinka - nearest hotel: No hotel found
borovskoy - nearest hotel: No hotel found
klyuchi - nearest hotel: No hotel found
anadyr - nearest hotel: No hotel found
stepnogorsk - nearest hotel: No hotel found
talnakh - nearest hotel: No hotel found
amga - nearest hotel: No hotel found
chunskiy - nearest hotel: No hotel found
berdigestyakh - nearest hotel: No hotel found
torghay - nearest hotel: No hotel found
urengoy - nearest hotel: No hotel found
kyren - nearest hotel: No hotel found
pangody - nearest hotel: No hotel found
nikolayevsk-on-amure - nearest hotel: No hotel found
beloyarskiy - nearest hotel: No hotel found
chany - nearest hotel: No hotel found
kargasok - nearest hotel: No hotel found
pyt-yakh - nearest hotel: No hotel found
yermakovskoye - nearest hotel: No hotel found
strezhevoy - nearest hote

,City,Country,Lat,Lng,Humidity,Hotel Name
15,tazovsky,RU,67.4667,78.7000,95,No hotel found
24,udachny,RU,66.4167,112.4000,97,No hotel found
26,jalai nur,CN,49.4500,117.7000,98,No hotel found
42,erdenet,MN,49.0333,104.0833,100,No hotel found
56,dudinka,RU,69.4058,86.1778,98,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [38]:
# Configure the map plot 
map_plot = hotel_df.hvplot.scatter(
    x='Lng',
    y='Lat',
    c='Humidity',
    cmap='viridis',
    title='City Map and Hotels',
    colorbar=True,
    size='Humidity',
    frame_width=700,
    frame_height=500,
    hover_cols=['City', 'Country', 'Humidity', 'Hotel Name'])

# Display the map
map_plot


:Scatter   [Lng]   (Lat,Humidity,City,Country,Hotel Name)